In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, StratifiedKFold
import os

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler

%matplotlib inline

In [ ]:
# import data
data_df_1 = pd.read_csv("../../data/training_validation_1.csv")
data_df_2 = pd.read_csv("../../data/training_validation_2.csv")
data_df_3 = pd.read_csv("../../data/model_refinement.csv")
merged_df = pd.concat([data_df_1, data_df_2, data_df_3], axis=0) # Merge data frames
train_df = merged_df.copy() #merged_df.filter(regex="vCnt|value|class|run")



In [ ]:
sensor_list = list(train_df.columns)
sensor_list.remove('class')
len(sensor_list)

In [ ]:
from PIL import Image
import skimage.color

In [ ]:
d = "class_images_v2_i1"

def class_image(df):
    c, r = int(df.iloc[0]['class']), int(df.iloc[0]['run'])
    cs, rs = str(c), str(r)
    
    new_df = df.fillna(method='ffill').fillna(method='bfill')

#     df = df.filter(regex='vCnt|value').values
    df = df.filter(sensor_list).values

    img = np.transpose(MinMaxScaler().fit_transform(df))
    
    img1 = img.astype(np.uint8) * 255
    img2 = np.sin(2**0 * np.pi * img).astype(np.uint8) * 255
    img3 = np.cos(2**0 * np.pi * img).astype(np.uint8) * 255
    
    img4 = img.astype(np.uint8) * 255
    img5 = np.sin(2**1 * np.pi * img).astype(np.uint8) * 255
    img6 = np.cos(2**1 * np.pi * img).astype(np.uint8) * 255

    img789 = skimage.color.gray2rgb(img*255)

    result = np.zeros((3*img.shape[0], 1450, 3))
    
    n = img.shape[0]
    result[0:n, 0:img.shape[1], 0] = img1
    result[0:n, 0:img.shape[1], 1] = img2
    result[0:n, 0:img.shape[1], 2] = img3

    result[n:2*n, 0:img.shape[1], 0] = img4
    result[n:2*n, 0:img.shape[1], 1] = img5
    result[n:2*n, 0:img.shape[1], 2] = img6
    
    result[2*n:, 0:img.shape[1]] = img789[:,:]

    im = Image.fromarray(result.astype('uint8'), 'RGB')
    
    im.save(os.path.join(d, "_".join(["class", cs, "run", rs])) + ".png")


In [ ]:
class_run_df = train_df[['class', 'run']].drop_duplicates()

In [ ]:
import gc

for c, r in class_run_df.values:
    print (c, r)
    df = train_df[(train_df['class'] == c) & (train_df['run'] == r)]
    class_image(df)
    
    gc.collect()
    
